I rapidly prototyped several machine learning models (gradient boosting, Xgboost, random forests, logistic regression) and noted that gradient boosting achieved the best results. This notebook outlines a mechanism for training the model and generating outputs and should be seen as indicative  - but this is not final code (there is little in the way of checks on missing / incorrectly formatted data etc).

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.ensemble import GradientBoostingClassifier  
from sklearn import cross_validation, metrics   
from sklearn.grid_search import GridSearchCV   

Load up the input and test files. Paths are hardcoded, and the files are assumed to be in the working directory.

In [ ]:
ml_dataset = pd.read_csv("train.csv")
evaluation_set = pd.read_csv("test.csv")



In [ ]:
# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['TARGET'].isnull()]


Fill missing values in predictors with column means.

In [ ]:
#Choose all predictors except target in both the 
predictors = [x for x in ml_dataset.columns if x not in ['class_col']]
for feature in predictors:
    v = ml_dataset[feature].mean()
    ml_dataset[feature] = ml_dataset[feature].fillna(v)
    evaluation_set[feature] = evaluation_set[feature].fillna(v)
    print('Imputed missing values in feature %s with value %s' % (feature, str(v)))


Train a model with *some* of the parameters tuned using grid search. With more time, some of the other variables could be tuned too. I also evaluated logistic regression, xgboost, SVM, NNs and random forests and got best results with gradient boosting (XGBoost had more variance during cross validation search, suggesting greater sensitivity to parameters). 

In [ ]:
params = {'max_depth':list(range(3,18,3)), 'min_samples_leaf':list(range(3,15,3)), 'min_samples_split':list(range(2,502,50)),'n_estimators':list(range(20,81,10))}
gsearch = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, max_features='sqrt',random_state=10), 
param_grid = params, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch.fit(ml_dataset[predictors],ml_dataset['TARGET'])

Train the final model and generate results.

In [ ]:
finalmodel = gsearch.best_estimator_.fit(ml_dataset[predictors],ml_dataset['TARGET'])

In [ ]:
#generate predictions (just to see things look okay) and probabilities [yhat]
predictions = finalmodel.predict(evaluation_set[predictors])
predprob = finalmodel.predict_proba(evaluation_set[predictors])[:,1]

In [ ]:
# results = pd.DataFrame(predprob, columns=['yhat'])
results.to_csv("results.csv")